- **Nama:** Aulia Afifah
- **ID Dicoding:** auliaafifah253
- Kaggle: https://www.kaggle.com/datasets/ibamibrahim/indonesian-news-title

In [1]:
import pandas as pd
df = pd.read_csv('indonesian-news-title.csv')
df = df.drop(columns=['date', 'url'])

In [2]:
df.head()

,title,category
0,Kemnaker Awasi TKA di Meikarta,finance
1,BNI Digitalkan BNI Java Jazz 2020,finance
2,"Terbang ke Australia, Edhy Prabowo Mau Genjot ...",finance
3,OJK Siapkan Stimulus Ekonomi Antisipasi Dampak...,finance
4,Saran Buat Anies-RK yang Mangkir Rapat Banjir ...,finance


In [3]:
category = pd.get_dummies(df.category)
df_baru = pd.concat([df, category], axis=1)
df_baru = df_baru.drop(columns='category')
df_baru

,title,finance,food,health,hot,inet,news,oto,sport,travel
0,Kemnaker Awasi TKA di Meikarta,True,False,False,False,False,False,False,False,False
1,BNI Digitalkan BNI Java Jazz 2020,True,False,False,False,False,False,False,False,False
2,"Terbang ke Australia, Edhy Prabowo Mau Genjot ...",True,False,False,False,False,False,False,False,False
3,OJK Siapkan Stimulus Ekonomi Antisipasi Dampak...,True,False,False,False,False,False,False,False,False
4,Saran Buat Anies-RK yang Mangkir Rapat Banjir ...,True,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...
91012,"Ketumpahan Air Panas di Pesawat, Kamu Bisa Tun...",False,False,False,False,False,False,False,False,True
91013,Foto: Bali & 9 Destinasi Paling Instagramable ...,False,False,False,False,False,False,False,False,True
91014,Game Bikin Turis Ini Liburan ke Jepang untuk.....,False,False,False,False,False,False,False,False,True
91015,"Sekeluarga Didepak dari Pesawat, Maskapai Bila...",False,False,False,False,False,False,False,False,True


In [4]:
judul = df_baru['title'].values
label = df_baru[['finance', 'food', 'health', 'hot', 'inet', 'news', 'oto', 'sport', 'travel']].values

In [5]:
from sklearn.model_selection import train_test_split
judul_latih, judul_test, label_latih, label_test = train_test_split(judul, label, test_size=0.2)

In [6]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=5000, oov_token='x')
tokenizer.fit_on_texts(judul_latih)

sekuens_latih = tokenizer.texts_to_sequences(judul_latih)
sekuens_test = tokenizer.texts_to_sequences(judul_test)

padded_latih = pad_sequences(sekuens_latih)
padded_test = pad_sequences(sekuens_test)

In [7]:
import tensorflow as tf
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=5000, output_dim=16),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(9, activation='softmax')
])
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [8]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>0.9):
      print("\nAkurasi telah mencapai >90%!")
      self.model.stop_training = True
callbacks = myCallback()

In [9]:
num_epochs = 30
history = model.fit(padded_latih, label_latih, epochs=num_epochs,
                    validation_data=(padded_test, label_test), verbose=2,callbacks=[callbacks])

Epoch 1/30
2276/2276 - 36s - loss: 1.1054 - accuracy: 0.6212 - val_loss: 0.7229 - val_accuracy: 0.7671 - 36s/epoch - 16ms/step
Epoch 2/30
2276/2276 - 31s - loss: 0.6855 - accuracy: 0.7811 - val_loss: 0.6563 - val_accuracy: 0.7915 - 31s/epoch - 13ms/step
Epoch 3/30
2276/2276 - 31s - loss: 0.6062 - accuracy: 0.8116 - val_loss: 0.6134 - val_accuracy: 0.8062 - 31s/epoch - 14ms/step
Epoch 4/30
2276/2276 - 31s - loss: 0.5449 - accuracy: 0.8318 - val_loss: 0.5957 - val_accuracy: 0.8195 - 31s/epoch - 14ms/step
Epoch 5/30
2276/2276 - 31s - loss: 0.4956 - accuracy: 0.8478 - val_loss: 0.5698 - val_accuracy: 0.8242 - 31s/epoch - 14ms/step
Epoch 6/30
2276/2276 - 30s - loss: 0.4551 - accuracy: 0.8600 - val_loss: 0.5619 - val_accuracy: 0.8240 - 30s/epoch - 13ms/step
Epoch 7/30
2276/2276 - 30s - loss: 0.4231 - accuracy: 0.8693 - val_loss: 0.5710 - val_accuracy: 0.8279 - 30s/epoch - 13ms/step
Epoch 8/30
2276/2276 - 31s - loss: 0.3956 - accuracy: 0.8774 - val_loss: 0.5628 - val_accuracy: 0.8304 - 31s/ep